In [1]:
## trying to build a CNN to be used in the RNN

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os

# Viz
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Feature Engineering
from sklearn.preprocessing import StandardScaler

# Models
from sklearn.linear_model import LinearRegression, LassoCV, Lasso

# Model Selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV

# Model Storage
from sklearn.externals import joblib

# CNN
import keras as k
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

# Image 
from PIL import Image

Using TensorFlow backend.


In [3]:
## get the wide data
data = pd.read_csv('C:/Users/Hans/Documents/CalPoly4thYear/Data451/complete_wide.csv')

In [4]:
## creating variables for x, y, z variables
x = []
y = []
z = []
for i in range(2,242,3):
    x.append('X' + str(i))
    y.append('X' + str(i+1))
    z.append('X' + str(i+2))

In [26]:
three_dim_data = np.array([np.array(data[x]),np.array(data[y]),np.array(data[z])])

In [27]:
data

,Unnamed: 0,Unnamed: 0.1,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X121,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X188,X189,X190,X191,X192,X193,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,Unnamed: 0.2,observation,date,activity,coding,time
0,0,1.0,2017-10-06 16:44:00,-0.191,-0.711,-0.660,-0.172,-0.715,-0.652,-0.207,-0.707,-0.656,-0.223,-0.719,-0.703,-0.246,-0.730,-0.727,-0.227,-0.691,-0.738,-0.207,-0.691,-0.719,-0.203,-0.711,-0.684,-0.211,-0.699,-0.668,-0.215,-0.684,-0.656,-0.195,-0.680,-0.652,-0.223,-0.703,-0.648,-0.305,-0.754,-0.668,-0.379,-0.773,-0.676,-0.395,-0.777,-0.691,-0.367,-0.746,-0.672,-0.316,-0.730,-0.641,-0.289,-0.727,-0.629,-0.297,-0.719,-0.625,-0.297,-0.734,-0.621,-0.273,-0.742,-0.625,-0.285,-0.742,-0.641,-0.359,-0.738,-0.652,-0.359,-0.734,-0.672,-0.316,-0.727,-0.652,-0.293,-0.734,-0.629,-0.301,-0.758,-0.621,-0.289,-0.754,-0.617,-0.266,-0.738,-0.613,-0.281,-0.730,-0.617,-0.301,-0.734,-0.637,-0.316,-0.750,-0.652,-0.332,-0.746,-0.656,-0.320,-0.738,-0.648,-0.320,-0.730,-0.625,-0.266,-0.715,-0.613,-0.254,-0.742,-0.621,-0.324,-0.766,-0.629,-0.328,-0.777,-0.621,-0.305,-0.789,-0.613,-0.270,-0.781,-0.602,-0.301,-0.738,-0.609,-0.285,-0.750,-0.609,-0.297,-0.754,-0.598,-0.305,-0.738,-0.590,-0.305,-0.727,-0.598,-0.324,-0.707,-0.613,-0.328,-0.727,-0.605,-0.332,-0.762,-0.594,-0.328,-0.770,-0.605,-0.328,-0.758,-0.621,-0.328,-0.754,-0.625,-0.328,-0.766,-0.633,-0.359,-0.770,-0.602,-0.305,-0.742,-0.570,-0.258,-0.754,-0.578,-0.293,-0.746,-0.605,-0.293,-0.738,-0.609,-0.305,-0.750,-0.625,-0.309,-0.691,-0.590,-0.316,-0.723,-0.582,-0.309,-0.773,-0.590,-0.270,-0.789,-0.602,-0.301,-0.777,-0.637,-0.316,-0.773,-0.652,-0.324,-0.766,-0.637,-0.336,-0.758,-0.633,-0.336,-0.754,-0.645,-0.340,-0.742,-0.652,-0.332,-0.730,-0.648,-0.313,-0.746,-0.645,-0.297,-0.750,-0.660,-0.281,-0.730,-0.672,-0.242,-0.723,-0.660,-0.223,-0.707,-0.652,-0.238,-0.719,-0.656,-0.273,-0.730,-0.652,-0.297,-0.742,-0.652,-0.309,-0.738,-0.648,-0.281,-0.730,-0.648,1.0,H_AM01DO1_M,13-01-2018,active,non-sedentary,0.0
1,1,2.0,2017-10-06 16:44:01,-0.254,-0.715,-0.645,-0.266,-0.727,-0.637,-0.289,-0.750,-0.637,-0.234,-0.734,-0.641,-0.195,-0.742,-0.645,-0.188,-0.730,-0.637,-0.227,-0.746,-0.652,-0.277,-0.738,-0.668,-0.289,-0.734,-0.672,-0.273,-0.762,-0.668,-0.277,-0.742,-0.680,-0.234,-0.738,-0.660,-0.184,-0.734,-0.676,-0.215,-0.734,-0.691,-0.238,-0.734,-0.691,-0.270,-0.711,-0.688,-0.273,-0.707,-0.695,-0.219,-0.742,-0.699,-0.188,-0.738,-0.707,-0.199,-0.727,-0.719,-0.199,-0.715,-0.719,-0.172,-0.742,-0.723,-0.191,-0.754,-0.742,-0.207,-0.738,-0.762,-0.199,-0.711,-0.770,-0.195,-0.676,-0.777,-0.164,-0.664,-0.793,-0.203,-0.660,-0.781,-0.203,-0.691,-0.758,-0.141,-0.707,-0.734,-0.043,-0.688,-0.715,0.020,-0.680,-0.727,-0.027,-0.672,-0.730,-0.035,-0.672,-0.730,-0.035,-0.668,-0.730,-0.020,-0.672,-0.730,-0.016,-0.676,-0.730,-0.047,-0.691,-0.742,-0.039,-0.719,-0.707,0.004,-0.711,-0.695,0.027,-0.703,-0.723,0.012,-0.691,-0.742,-0.020,-0.688,-0.742,-0.035,-0.688,-0.738,-0.063,-0.680,-0.742,-0.090,-0.699,-0.742,-0.094,-0.688,-0.742,-0.113,-0.703,-0.746,-0.125,-0.699,-0.746,-0.102,-0.691,

In [9]:
## building a pseudo training and testing set just to get going
## create x-var list
xvars = []
for i in range(2, 242):
    var = 'X' + str(i)
    xvars.append(var)
yvars = ['activity']

train = data.sample(int(0.7 * len(data)))
test = data.sample(int(0.7 * len(data)))
x_train = train[xvars]
x_test = test[xvars]
y_train = train[yvars]
y_test = test[yvars]

In [10]:
x_train.values

array([[ 0.938, -0.277,  0.207, ...,  0.957, -0.313,  0.023],
       [-0.563,  0.328, -0.492, ...,  0.254,  0.543, -0.34 ],
       [-0.234, -0.328,  0.883, ..., -0.234, -0.324,  0.879],
       ..., 
       [ 0.695, -0.27 , -0.652, ...,  0.668, -0.141, -0.668],
       [-0.383, -1.043, -0.637, ..., -0.27 , -1.246,  0.008],
       [-0.203,  0.73 , -0.418, ...,  0.098,  0.559, -0.527]])

In [11]:
x_train.shape

(70283, 240)

# THE HUMAN ACTIVITY ONE

In [60]:
data_small = data[data['observation'] == 'H_AM01DO1_M']

In [26]:
## reshape the data to match the example data code
X = data[x].values.flatten()
Y = data[y].values.flatten()
Z = data[z].values.flatten()
acts = np.repeat(data['activity'],80)

# my_data = np.dstack([X,Y,Z])
my_data = pd.DataFrame({'X':X,
                        'Y':Y,
                        'Z':Z,
                       'activity':acts})

,X,Y,Z,activity
0,-0.191,-0.711,-0.660,active
0,-0.172,-0.715,-0.652,active
0,-0.207,-0.707,-0.656,active
0,-0.223,-0.719,-0.703,active
0,-0.246,-0.730,-0.727,active
0,-0.227,-0.691,-0.738,active
0,-0.207,-0.691,-0.719,active
0,-0.203,-0.711,-0.684,active
0,-0.211,-0.699,-0.668,active
0,-0.215,-0.684,-0.656,active


0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
0            0.0
           ...  
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200.0
100404    7200

In [16]:
data.tail()

,Unnamed: 0,Unnamed: 0.1,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,X79,X80,X81,X82,X83,X84,X85,X86,X87,X88,X89,X90,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100,X101,X102,X103,X104,X105,X106,X107,X108,X109,X110,X111,X112,X113,X114,X115,X116,X117,X118,X119,X120,X121,X122,X123,X124,X125,X126,X127,X128,X129,X130,X131,X132,X133,X134,X135,X136,X137,X138,X139,X140,X141,X142,X143,X144,X145,X146,X147,X148,X149,X150,X151,X152,X153,X154,X155,X156,X157,X158,X159,X160,X161,X162,X163,X164,X165,X166,X167,X168,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178,X179,X180,X181,X182,X183,X184,X185,X186,X187,X188,X189,X190,X191,X192,X193,X194,X195,X196,X197,X198,X199,X200,X201,X202,X203,X204,X205,X206,X207,X208,X209,X210,X211,X212,X213,X214,X215,X216,X217,X218,X219,X220,X221,X222,X223,X224,X225,X226,X227,X228,X229,X230,X231,X232,X233,X234,X235,X236,X237,X238,X239,X240,X241,Unnamed: 0.2,observation,date,activity,coding,time
100400,7196,7197.0,2017-09-23 11:12:56,0.984,0.191,-0.047,0.980,0.188,-0.047,0.988,0.176,-0.051,0.996,0.172,-0.051,0.996,0.168,-0.051,1.000,0.164,-0.047,1.000,0.168,-0.043,1.000,0.164,-0.043,1.000,0.164,-0.047,0.988,0.164,-0.047,0.984,0.176,-0.051,0.980,0.176,-0.051,0.973,0.172,-0.055,0.973,0.184,-0.063,0.984,0.191,-0.066,0.996,0.195,-0.063,0.996,0.199,-0.055,1.008,0.199,-0.055,1.012,0.195,-0.043,1.012,0.195,-0.047,1.004,0.199,-0.055,0.996,0.199,-0.063,0.988,0.199,-0.074,0.984,0.199,-0.082,0.984,0.191,-0.086,0.988,0.188,-0.082,0.992,0.180,-0.074,1.000,0.180,-0.070,1.008,0.168,-0.066,1.023,0.164,-0.066,1.031,0.164,-0.066,1.035,0.164,-0.066,1.031,0.160,-0.074,1.008,0.160,-0.082,0.992,0.156,-0.098,0.973,0.145,-0.105,0.965,0.133,-0.113,0.973,0.137,-0.109,0.977,0.137,-0.105,0.984,0.129,-0.102,0.977,0.133,-0.105,0.969,0.102,-0.109,0.957,0.094,-0.113,0.945,0.086,-0.121,0.941,0.082,-0.129,0.938,0.074,-0.129,0.938,0.070,-0.121,0.949,0.066,-0.121,0.969,0.066,-0.125,0.992,0.070,-0.129,1.000,0.074,-0.133,1.004,0.078,-0.133,1.000,0.082,-0.125,1.000,0.094,-0.133,1.000,0.105,-0.133,1.008,0.113,-0.145,1.008,0.117,-0.152,1.008,0.125,-0.160,1.012,0.129,-0.168,1.012,0.137,-0.172,1.012,0.145,-0.184,1.008,0.152,-0.191,1.004,0.160,-0.199,0.996,0.164,-0.207,0.988,0.168,-0.207,0.973,0.168,-0.211,0.965,0.172,-0.211,0.953,0.168,-0.207,0.938,0.164,-0.219,0.938,0.164,-0.223,0.934,0.168,-0.230,0.914,0.172,-0.223,0.902,0.168,-0.211,0.910,0.164,-0.203,0.926,0.164,-0.199,0.945,0.164,-0.195,0.949,0.168,-0.184,0.953,0.168,-0.172,0.941,0.164,-0.168,0.938,0.168,-0.176,7197.0,E_AM14DO2_M,21-01-2018,active,non-sedentary,7196.0
100401,7197,7198.0,2017-09-23 11:12:57,0.961,0.168,-0.176,0.945,0.164,-0.168,0.949,0.156,-0.168,0.941,0.164,-0.137,0.965,0.145,-0.125,0.984,0.129,-0.125,0.996,0.117,-0.141,1.000,0.121,-0.137,1.000,0.137,-0.145,1.008,0.145,-0.152,0.992,0.145,-0.160,0.988,0.145,-0.160,0.988,0.148,-0.148,0.969,0.156,-0.137,0.949,0.160,-0.137,0.945,0.168,-0.137,0.941,0.176,-0.137,0.938,0.184,-0.145,0.949,0.195,-0.137,0.957,0.199,-0.137,0.957,0.199,-0.156,0.953,0.223,-0.137,0.887,0.258,-0.148,0.902,0.266,-0.109,1.066,0.246,-0.051,1.156,0.234,-0.004,1.156,0.242,0.008,1.176,0.238,-0.004,1.207,0.219,0.008,1.172,0.203,0.027,1.164,0.168,0.055,1.203,0.148,0.070,1.195,0.152,0.094,1.176,0.172,0.102,1.180,0.164,0.094,1.141,0.109,0.082,1.094,0.070,0.074,1.039,0.074,0.074,0.969,0.059,0.082,1.008,0.066,0.105,1.090,0.078,0.109,1.109,0.043,0.098,1.145,0.047,0.094,1.113,0.016,0.105,1.063,0.004,0.113,1.102,0.000,0.105,1.117,-0.043,0.070,1.070,-0.059,0.063,1.063,-0.059,0.059,1.078,-0.098,0.051,1.004,-0.137,0.039,0.953,-0.145,0.043,0.922,-0.164,0.043,0.934,-0.188,0.031,0.910,-0.180,0.027,0.945,-0.172,0.004,1.070,-0.180,-0.023,1.066,-0.168,-0.059,0.988,-0.180,-0.105,0.898,-0.195,-0.148,0.840,-0.215,-0.164,0.852,-0.254,-0.172,0.848,

In [47]:
# dataset = read_data('actitracker_raw.txt')
# dataset.dropna(axis=0, how='any', inplace= True)
# dataset['x-axis'] = feature_normalize(dataset['x-axis'])
# dataset['y-axis'] = feature_normalize(dataset['y-axis'])
# dataset['z-axis'] = feature_normalize(dataset['z-axis'])

In [139]:
from scipy import stats
import tensorflow as tf

## not me
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += 1

## not me
def segment_signal(data,window_size = 10):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data["time"], window_size):
        X = data[x][start:end]
        Y = data[y][start:end]
        Z = data[z][start:end]
#         return X,Y,Z
        if(len(data["X1"][start:end]) == window_size):
            print(X.shape)
            print(Y.shape)
            print(Z.shape)
            segments = np.vstack([segments,np.dstack([X,Y,Z])])
            labels = np.append(labels,stats.mode(data["activity"][start:end])[0][0])
    return segments, labels

def make_windows(data,size):
    """
    this function will create the windows out of the 240 values
    from one second.
    Will be required to be used in a loop
    :param data: one row of the data from
    :param size: window size we want to use
    :return list of indices to start and stop for the windows
    """
    start = range(80 - size)
    stop = range(size - 1, 80)
    return start,stop

def make_segments(data,window_size=10):
    """
    :param data: the wide dataframe (this function will deal with iterations)
    :param window_size: how many 1/80 of seconds we want to convolve
    :return pieces to convolve, then the activity type associated with each of them
    """
    ## create an empty array to hold x,y,z convolving values
    segments = np.empty((0,window_size,3))
    ## create array to hold the activity levels
    labels = np.empty((0))
    ## create the starting and stop spots
    start,stop = make_windows(data,window_size)
    ## create segments and labels (looping through rows to do so)
    for index,row in data.iterrows():
        ## for each of the rows (each second), add all the segments
        for i in start:
            ## for each of the directions, grab all data
            X = np.array(row[x][i:i+window_size])
            Y = np.array(row[y][i:i+window_size])
            Z = np.array(row[z][i:i+window_size])
            ## dimensionally stack the X,Y,Z            
            segments = np.vstack([segments,np.dstack([X,Y,Z])])
            labels = np.append(labels,row['activity'])
    return segments,labels

In [ ]:
gus,yash = make_segments(data,10)

In [41]:
segments, labels = make_segments(data,10)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
reshaped_segments = segments.reshape(len(segments), 1, 2, 3)

(2, 80)
(2, 80)
(2, 80)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
## Split training and testing set
train_test_split = np.random.rand(len(reshaped_segments)) < 0.70
train_x = reshaped_segments[train_test_split]
train_y = labels[train_test_split]
test_x = reshaped_segments[~train_test_split]
test_y = labels[~train_test_split]

## From Kaggle

In [5]:
nb_features = 64
nb_class = 6

# Create our model, will tune these / try different architectures as the week goes on
model = Sequential()

model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

C:\Users\Hans\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(64, 3), filters=512, kernel_size=1)`
  import sys


In [6]:
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [37]:
model.fit(x_train, y_train, nb_epoch=15, validation_data=(x_test, y_test), batch_size=16)

C:\Users\Hans\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected conv1d_3_input to have 3 dimensions, but got array with shape (70283, 240)

## A different CNN on Kaggle

In [39]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.initializers import random_uniform

import pandas as pd




In [45]:
x_train.shape

(70283, 240)

In [46]:
#hyperparameters
input_dimension = 226
learning_rate = 0.0025
momentum = 0.85
hidden_initializer = random_uniform(seed=6)
dropout_rate = 0.2


# create model
model = Sequential()
## changed x_train.shape[1:3] -> x_train.shape[1]
model.add(Convolution1D(nb_filter=32, filter_length=3, input_shape=x_train.shape[1], activation='relu'))
model.add(Convolution1D(nb_filter=16, filter_length=1, activation='relu'))
model.add(Flatten())
model.add(Dropout(dropout_rate))
model.add(Dense(128, input_dim=input_dimension, kernel_initializer=hidden_initializer, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(64, kernel_initializer=hidden_initializer, activation='relu'))
model.add(Dense(2, kernel_initializer=hidden_initializer, activation='softmax'))

sgd = SGD(lr=learning_rate, momentum=momentum)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['acc'])
model.fit(x_train, y_train, epochs=5, batch_size=128)
predictions = model.predict_proba(x_test)

ans = pd.DataFrame(predictions)
ans = ans[0]

C:\Users\Hans\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=240, activation="relu", filters=32, kernel_size=3)`
  del sys.path[0]


TypeError: 'int' object is not iterable

## Trying sklearn's

In [79]:
from sknn.mlp import Convolution,Classifier, Layer

N = 240

nn = Classifier(
    layers=[
        Convolution("Rectifier", kernel_shape=(N,1)),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=5)
nn.fit(x_train.values, y_train.values)

# nn = Classifier(
#     layers=[
#         Layer("Rectifier", units=100),
#         Layer("Linear")],
#     learning_rate=0.02,
#     n_iter=10)
# nn.fit(x_train.values, y_train.values)

# y_valid = nn.predict(X_valid)

# score = nn.score(X_test, y_test)

AssertionError: Input array is not in image shape, and could not assume a square.

In [65]:
x_train.values[0].shape

(240,)

## LET'S TRY ANOTHER EXAMPLE CUZ FUCK IT